# Tutorial 01 - Thermal Block

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.fem
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import slepc4py  # noqa: F401
import ufl

In [ ]:
import rbnicsx.backends
import rbnicsx.online

In [ ]:
%load_ext nbvalx

In [ ]:
%register_run_if_allowed_tags \
    reduced_basis_inefficient, reduced_basis_efficient, \
    pod_galerkin_inefficient, pod_galerkin_efficient, \
    pod_neural_networks_jax

In [ ]:
%register_run_if_current_tag reduced_basis_inefficient

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
gmsh.initialize()
gmsh.model.add("thermal_block")

In [ ]:
p0 = gmsh.model.geo.addPoint(-1.0, -1.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, -1.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(-1.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
outer_boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])

In [ ]:
p4 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p5 = gmsh.model.geo.addPoint(0.5, 0.0, 0.0, mesh_size)
p6 = gmsh.model.geo.addPoint(-0.5, 0.0, 0.0, mesh_size)
c0 = gmsh.model.geo.addCircleArc(p5, p4, p6)
c1 = gmsh.model.geo.addCircleArc(p6, p4, p5)
interface = gmsh.model.geo.addCurveLoop([c0, c1])

In [ ]:
outer_subdomain = gmsh.model.geo.addPlaneSurface([outer_boundary, interface])
inner_subdomain = gmsh.model.geo.addPlaneSurface([interface])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1, l3], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(2, [inner_subdomain], 1)
gmsh.model.addPhysicalGroup(2, [outer_subdomain], 2)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        self._trial = u
        self._test = v
        # Define measures for integration of forms
        dx = ufl.Measure("dx")(subdomain_data=subdomains)
        ds = ufl.Measure("ds")(subdomain_data=boundaries)
        self._dx = dx
        self._ds = ds
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = mu_symb[0] * ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(1) + ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(2)
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        f = ufl.inner(mu_symb[1], v) * ds(1)
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        zero = petsc4py.PETSc.ScalarType(0)
        facets_top = boundaries.indices[boundaries.values == 3]
        bdofs_V_top = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_top)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V_top, V)]
        self._bcs = bcs

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def mu_symb(self) -> rbnicsx.backends.SymbolicParameters:
        """Return the symbolic parameters of the problem."""
        return self._mu_symb

    @property
    def trial_and_test(self) -> typing.Tuple[ufl.Argument, ufl.Argument]:
        """Return the UFL arguments used in the construction of the forms."""
        return self._trial, self._test

    @property
    def measures(self) -> typing.Tuple[ufl.Measure, ufl.Measure]:
        """Return the UFL measures used in the construction of the forms."""
        return self._dx, self._ds

    @property
    def bilinear_form(self) -> ufl.Form:
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.Function(self._V)
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([8.0, -1.0])
solution = problem.solve(mu_solve)

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution")

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
class StabilityFactorEvaluation(object):
    """Define the eigenvalue problem to compute the coercivity constant, and solve it with EPS."""

    def __init__(self, problem: problem) -> None:
        # Get function space from the high fidelity problem
        V = problem.function_space
        self._V = V
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Get measures from the high fidelity problem
        dx, _ = problem.measures
        # Store symbolic parameters from problem
        self._mu_symb = problem.mu_symb
        # Define bilinear forms of the eigenvalue problem
        a = problem.bilinear_form
        b = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        self._b = a
        self._b_cpp = dolfinx.fem.form(b)
        # Define restriction that throws away DOFs associated to Dirichlet boundary conditions
        bcs = problem.boundary_conditions
        bcs_dof_indices = np.hstack([bc.dof_indices()[0] for bc in bcs])
        dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
        restriction = multiphenicsx.fem.DofMapRestriction(V.dofmap, np.setdiff1d(dofs_V, bcs_dof_indices))
        self._restriction = restriction

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the eigenvalue problem."""
        return self._V

    @property
    def bilinear_form_left(self) -> ufl.Form:
        """Return the bilinear form of the left-hand side of the eigenvalue problem."""
        return self._a

    @property
    def bilinear_form_right(self) -> ufl.Form:
        """Return the bilinear form of the right-hand side of the eigenvalue problem."""
        return self._f

    @property
    def restriction(self) -> multiphenicsx.fem.DofMapRestriction:
        """Return restriction that throws away DOFs associated to homogenous Dirichlet boundary conditions."""
        return self._restriction

    def _assemble_matrix_left(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = multiphenicsx.fem.assemble_matrix(self._a_cpp, restriction=(self._restriction, self._restriction))
        A.assemble()
        return A

    def _assemble_matrix_right(self) -> petsc4py.PETSc.Mat:
        """Assemble the right-hand side matrix."""
        B = multiphenicsx.fem.assemble_matrix(self._b_cpp, restriction=(self._restriction, self._restriction))
        B.assemble()
        return B

    def evaluate(self, mu: np.typing.NDArray[np.float64]) -> petsc4py.PETSc.RealType:
        """Assign the provided parameters value and solve the eigenvalue problem."""
        self._mu_symb.value[:] = mu
        return self._evaluate()

    def _evaluate(self) -> petsc4py.PETSc.RealType:
        """Solve the eigenvalue problem with EPS."""
        A = self._assemble_matrix_left()
        B = self._assemble_matrix_right()
        eps = slepc4py.SLEPc.EPS().create(mesh.comm)
        eps.setOperators(A, B)
        eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GNHEP)
        eps.setDimensions(1, petsc4py.PETSc.DECIDE, petsc4py.PETSc.DECIDE)
        eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
        eps.setTarget(1.e-5)
        eps.getST().setType(slepc4py.SLEPc.ST.Type.SINVERT)
        eps.getST().getKSP().setType("preonly")
        eps.getST().getKSP().getPC().setType("lu")
        eps.getST().getKSP().getPC().setFactorSolverType("mumps")
        eps.solve()
        assert eps.getConverged() >= 1
        eigv = eps.getEigenvalue(0)
        r, i = eigv.real, eigv.imag
        assert np.isclose(i, 0.0)
        return r

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
stability_factor_evaluation = StabilityFactorEvaluation(problem)

In [ ]:
%%run_if reduced_basis_inefficient
stability_factor_lower_bound = stability_factor_evaluation

In [ ]:
%%run_if reduced_basis_inefficient
stability_factor = stability_factor_evaluation.evaluate(mu_solve)
print(f"Stability factor at {mu_solve} is {stability_factor}")

## 3. Reduced problem definition

In [ ]:
%%run_if reduced_basis_efficient
class StabilityFactorLowerBound(object):
    """Evaluate a lower bound of the stability factor using the min-theta method."""

    def evaluate(self, mu: np.typing.NDArray[np.float64]) -> petsc4py.PETSc.RealType:
        """Evaluate the stability factor using the min-theta method."""
        return min(mu[0], 1)

In [ ]:
%%run_if reduced_basis_efficient
stability_factor_lower_bound = StabilityFactorLowerBound()

In [ ]:
%%run_if reduced_basis_efficient
stability_factor_exact = stability_factor_evaluation.evaluate(mu_solve)
stability_factor_min_theta = stability_factor_lower_bound.evaluate(mu_solve)
print(f"Stability factor at {mu_solve} is {stability_factor_exact}")
print(f"Min-theta approximation of the stability factor at {mu_solve} is {stability_factor_min_theta}")
assert (
    stability_factor_min_theta <= stability_factor_exact
    or np.isclose(stability_factor_min_theta, stability_factor_exact))

In [ ]:
%%run_if pod_galerkin_inefficient, reduced_basis_inefficient
class ProjectionBasedInefficientReducedProblem(object):
    """Define a linear projection-based problem, and solve it with KSP."""

    def __init__(self, problem: Problem) -> None:
        # Define basis functions storage
        basis_functions = rbnicsx.backends.FunctionsList(problem.function_space)
        self._basis_functions = basis_functions
        # Store symbolic parameters from problem
        self._mu_symb = problem.mu_symb
        # Store forms of the problem
        self._a = problem.bilinear_form
        self._f = problem.linear_form
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Define inner product using the H^1 seminorm.
        inner_product = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
        self._inner_product = inner_product
        self._inner_product_action = rbnicsx.backends.bilinear_form_action(inner_product)
        # Define the energy inner product
        energy_inner_product = 0.5 * (problem._a + ufl.adjoint(problem._a))
        self._energy_inner_product = energy_inner_product
        self._energy_inner_product = rbnicsx.backends.bilinear_form_action(energy_inner_product)

    @property
    def basis_functions(self) -> rbnicsx.backends.FunctionsList:
        """Return the basis functions of the reduced problem."""
        return self._basis_functions

    @property
    def inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product

    @property
    def energy_inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the energy inner product associated to this reduced problem.

        For elliptic coercive problems this inner product is used to evaluate the norm of the error.
        """
        return self._energy_inner_product

    def _assemble_matrix(self, N: int) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side online matrix of dimension N."""
        return rbnicsx.backends.project_matrix(self._a, self._basis_functions[:N])

    def _assemble_vector(self, N: int) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side online vector of dimension N."""
        return rbnicsx.backends.project_vector(self._f, self._basis_functions[:N])

    def solve(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Assign the provided parameters value and solve the online problem."""
        self._mu_symb.value[:] = mu
        return self._solve(N)

    def _solve(self, N: int) -> petsc4py.PETSc.Vec:
        """Solve the linear online problem with KSP."""
        A = self._assemble_matrix(N)
        F = self._assemble_vector(N)
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.setFromOptions()
        reduced_solution = rbnicsx.online.create_vector(N)
        ksp.solve(F, reduced_solution)
        return reduced_solution

    def reconstruct_solution(self, reduced_solution: petsc4py.PETSc.Vec) -> dolfinx.fem.Function:
        """Reconstructed reduced solution on the high fidelity space."""
        return self.basis_functions[:reduced_solution.size] * reduced_solution

    def compute_pointwise_error(
        self, solution: dolfinx.fem.Function, reduced_solution: petsc4py.PETSc.Vec
    ) -> dolfinx.fem.Function:
        """Compute the pointwise error."""
        reconstructed_reduced_solution = self.reconstruct_solution(reduced_solution)
        assert solution.function_space == reconstructed_reduced_solution.function_space
        error = dolfinx.fem.Function(solution.function_space)
        error.vector[:] = solution.vector.array - reconstructed_reduced_solution.vector.array
        return error

    def compute_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], error: dolfinx.fem.Function
    ) -> petsc4py.PETSc.RealType:
        """Compute the error using the energy inner product."""
        self._mu_symb.value[:] = mu
        return np.sqrt(self._energy_inner_product(error, error))

In [ ]:
%%run_if pod_galerkin_inefficient
class PODGInefficientReducedProblem(ProjectionBasedInefficientReducedProblem):
    """Specialize the projection-based problem to basis generation with POD."""

    pass  # No further specialization needed

In [ ]:
%%run_if reduced_basis_inefficient
class RBInefficientReducedProblem(ProjectionBasedInefficientReducedProblem):
    """Specialize the projection-based problem to basis generation with RB."""

    def __init__(self, problem: Problem, stability_factor_lower_bound: object) -> None:
        super().__init__(problem)
        # Determine DOFs associated to Dirichlet boundary conditions
        self._bcs_dof_indices = np.hstack([bc.dof_indices()[0] for bc in problem.boundary_conditions])
        # Store object that evaluates the stability factor lower bound
        self._stability_factor_lower_bound = stability_factor_lower_bound

    def estimate_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate error as the ratio of the norm of the residual and the stability factor lower bound."""
        reconstructed_reduced_solution = self.reconstruct_solution(reduced_solution)
        u, _ = problem.trial_and_test
        residual_form = self._f - ufl.replace(self._a, {u: reconstructed_reduced_solution})
        residual_function = dolfinx.fem.Function(reconstructed_reduced_solution.function_space)
        dolfinx.fem.assemble_vector(residual_function.vector, dolfinx.fem.form(residual_form))
        residual_function.vector[self._bcs_dof_indices] = 0.0
        residual_function.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        return np.sqrt(self._inner_product_action(residual_function, residual_function)) / (
            self._stability_factor_lower_bound.evaluate(mu))

In [ ]:
%%run_if pod_galerkin_inefficient
reduced_problem = PODGInefficientReducedProblem(problem)

In [ ]:
%%run_if reduced_basis_inefficient
reduced_problem = RBInefficientReducedProblem(problem, stability_factor_lower_bound)

In [ ]:
%%run_if pod_galerkin_efficient, reduced_basis_efficient
class ProjectionBasedReducedProblem(object):
    """Define a linear projection-based problem, and solve it with KSP."""

    def __init__(self, problem: Problem) -> None:
        # Define basis functions storage
        basis_functions = rbnicsx.backends.FunctionsList(problem.function_space)
        self._basis_functions = basis_functions
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Get measures from the high fidelity problem
        dx, ds = problem.measures
        # Define separable expansion of the bilinear form of the high fidelity problem,
        # and storage for the corresponding pre-assembled reduced operators.
        thetas_a = (
            lambda mu: mu[0],
            lambda mu: 1
        )
        self._thetas_a = thetas_a
        operators_a = (
            ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(1),
            ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(2)
        )
        self._operators_a = operators_a
        assert len(thetas_a) == len(operators_a)
        self._Q_a = len(thetas_a)
        reduced_operators_a: typing.Dict[int, rbnicsx.backends.TensorsList] = dict()
        self._reduced_operators_a = reduced_operators_a
        # Define separable expansion of the linear form of the high fidelity problem,
        # and storage for the corresponding pre-assembled reduced operators.
        thetas_f = (
            lambda mu: mu[1],
        )
        self._thetas_f = thetas_f
        operators_f = (
            ufl.inner(1, v) * ds(1),
        )
        self._operators_f = operators_f
        assert len(thetas_f) == len(operators_f)
        self._Q_f = len(thetas_f)
        reduced_operators_f: typing.Dict[int, rbnicsx.backends.TensorsList] = dict()
        self._reduced_operators_f = reduced_operators_f
        # Define inner product using the H^1 seminorm.
        inner_product = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
        self._inner_product = inner_product
        self._inner_product_action = rbnicsx.backends.bilinear_form_action(inner_product)
        # Define the energy inner product
        energy_inner_product = 0.5 * (problem._a + ufl.adjoint(problem._a))
        self._energy_inner_product = energy_inner_product
        self._energy_inner_product = rbnicsx.backends.bilinear_form_action(energy_inner_product)

    @property
    def basis_functions(self) -> rbnicsx.backends.FunctionsList:
        """Return the basis functions of the reduced problem."""
        return self._basis_functions

    @property
    def inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product

    @property
    def energy_inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the energy inner product associated to this reduced problem.

        For elliptic coercive problems this inner product is used to evaluate the norm of the error.
        """
        return self._energy_inner_product

    def preassemble_reduced_operators(self, N: int) -> None:
        """Preassemble reduced operators in the separable expansion."""
        # Preassemble left-hand side reduced operators
        reduced_operators_a_N = rbnicsx.online.TensorsList((N, N))
        reduced_operators_a_N.extend([
            rbnicsx.backends.project_matrix(operator_a, self._basis_functions[:N])
            for operator_a in self._operators_a])
        assert N not in self._reduced_operators_a
        self._reduced_operators_a[N] = reduced_operators_a_N
        # Preassemble right-hand side reduced operators
        reduced_operators_f_N = rbnicsx.online.TensorsList(N)
        reduced_operators_f_N.extend([
            rbnicsx.backends.project_vector(operator_f, self._basis_functions[:N])
            for operator_f in self._operators_f])
        assert N not in self._reduced_operators_f
        self._reduced_operators_f[N] = reduced_operators_f_N

    def _assemble_matrix(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side online matrix of dimension N."""
        thetas_a_mu = rbnicsx.online.create_vector(self._Q_a)
        thetas_a_mu[:] = [theta_a(mu) for theta_a in self._thetas_a]
        return self._reduced_operators_a[N] * thetas_a_mu

    def _assemble_vector(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side online vector of dimension N."""
        thetas_f_mu = rbnicsx.online.create_vector(self._Q_f)
        thetas_f_mu[:] = [theta_f(mu) for theta_f in self._thetas_f]
        return self._reduced_operators_f[N] * thetas_f_mu

    def solve(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Solve the online problem."""
        A = self._assemble_matrix(mu, N)
        F = self._assemble_vector(mu, N)
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.setFromOptions()
        reduced_solution = rbnicsx.online.create_vector(N)
        ksp.solve(F, reduced_solution)
        return reduced_solution

    def reconstruct_solution(self, reduced_solution: petsc4py.PETSc.Vec) -> dolfinx.fem.Function:
        """Reconstructed reduced solution on the high fidelity space."""
        return self.basis_functions[:reduced_solution.size] * reduced_solution

    def compute_pointwise_error(
        self, solution: dolfinx.fem.Function, reduced_solution: dolfinx.fem.Function
    ) -> typing.Tuple[dolfinx.fem.Function, petsc4py.PETSc.RealType]:
        """Compute the pointwise error and its norm."""
        assert solution.function_space == reduced_solution.function_space
        error = dolfinx.fem.Function(solution.function_space)
        error.vector[:] = solution.vector.array - reduced_solution.vector.array
        return error

    def compute_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], error: dolfinx.fem.Function
    ) -> petsc4py.PETSc.RealType:
        """Compute the error using the energy inner product."""
        self._mu_symb.value[:] = mu
        return np.sqrt(self._energy_inner_product(error, error))

In [ ]:
%%run_if pod_galerkin_efficient
class PODGReducedProblem(ProjectionBasedReducedProblem):
    """Specialize the projection-based problem to basis generation with RB."""

    pass  # No further specialization needed

In [ ]:
%%run_if reduced_basis_efficient
class RBReducedProblem(ProjectionBasedReducedProblem):
    """Specialize the projection-based problem to basis generation with POD."""

    def __init__(self, problem: Problem, stability_factor_lower_bound: object) -> None:
        super().__init__(problem)
        # Determine DOFs associated to Dirichlet boundary conditions
        self._bcs_dof_indices = np.hstack([bc.dof_indices()[0] for bc in problem.boundary_conditions])
        # Store object that evaluates the stability factor lower bound
        self._stability_factor_lower_bound = stability_factor_lower_bound

    def estimate_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate error as the ratio of the norm of the residual and the stability factor lower bound."""
        pass  # TODO

In [ ]:
%%run_if pod_galerkin_efficient
reduced_problem = PODGReducedProblem(problem)

In [ ]:
%%run_if reduced_basis_efficient
reduced_problem = RBReducedProblem(problem, stability_factor_lower_bound)